In [1]:
!git clone https://github.com/imojlish/safeclinic.git
%cd safeclinic
     

[WinError 2] The system cannot find the file specified: 'safeclinic'
c:\Users\ISRAT\Downloads


'git' is not recognized as an internal or external command,
operable program or batch file.
C:\Users\ISRAT\AppData\Roaming\Python\Python313\site-packages\IPython\core\magics\osm.py:393: UserWarning: This is now an optional IPython functionality, using bookmarks requires you to install the `pickleshare` library.
  bkms = self.shell.db.get('bookmarks', {})


In [ ]:
import numpy as np
import cv2
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.models import load_model
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input

# Load the face detector and mask detector model
faceCascade = cv2.CascadeClassifier("haarcascade_frontalface_alt2.xml")
model = load_model("mask_recog.h5")

def face_mask_detector(frame):
    # Create a blurred version of the background
    blurred_frame = cv2.GaussianBlur(frame, (21, 21), 0)

    # Convert the frame to grayscale for face detection
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    faces = faceCascade.detectMultiScale(gray,
                                         scaleFactor=1.1,
                                         minNeighbors=5,
                                         minSize=(60, 60),
                                         flags=cv2.CASCADE_SCALE_IMAGE)
    
    # Create a mask for the face region
    mask = np.zeros_like(frame, dtype=np.uint8)
    
    # Process each detected face
    for (x, y, w, h) in faces:
        # Extract face region
        face_frame = frame[y:y+h, x:x+w]
        
        # Preprocess face for mask model
        face_frame_rgb = cv2.cvtColor(face_frame, cv2.COLOR_BGR2RGB)
        face_frame_resized = cv2.resize(face_frame_rgb, (224, 224))
        face_frame_array = img_to_array(face_frame_resized)
        face_frame_array = np.expand_dims(face_frame_array, axis=0)
        face_frame_array = preprocess_input(face_frame_array)
        
        # Predict mask or no mask
        preds = model.predict(face_frame_array)
        
        # Process prediction results
        for pred in preds:
            (mask_prob, withoutMask_prob) = pred  # Ensure the model gives two values per prediction
            print(f"Mask probability: {mask_prob}, No Mask probability: {withoutMask_prob}")  # Debug
            
            # Determine label based on probabilities
            if mask_prob > withoutMask_prob:
                label = "Mask"
                color = (0, 255, 0)  # Green for Mask
            else:
                label = "No Mask"
                color = (0, 0, 255)  # Red for No Mask

            # Format label with confidence
            confidence = max(mask_prob, withoutMask_prob) * 100
            label_text = "{}: {:.2f}%".format(label, confidence)
            
            # Draw label and bounding box on the frame
            cv2.putText(frame, label_text, (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.8, color, 2)
            cv2.rectangle(frame, (x, y), (x + w, y + h), color, 2)
            
            # Keep the face region in focus by copying it onto the mask
            mask[y:y+h, x:x+w] = face_frame  # Assign the face area to the mask
    
    # Combine blurred background and the mask to keep the face in focus
    output_frame = np.where(mask > 0, mask, blurred_frame)
    
    return output_frame

# Video processing function
def process_video(video_path):
    # Open the video file
    cap = cv2.VideoCapture(video_path)
    
    # Check if the video file opened successfully
    if not cap.isOpened():
        print("Error: Could not open video.")
        return
    
    # Process each frame in the video
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break  # Exit if no more frames are available
        
        # Detect face masks in the current frame
        output_frame = face_mask_detector(frame)
        
        # Display the frame (in Colab use cv2_imshow)
        cv2_imshow(output_frame)  # For Colab
        # cv2.imshow("Face Mask Detection", output_frame)  # Uncomment for local execution
        
        # Exit if 'q' is pressed
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break
    
    # Release the video capture object and close display window
    cap.release()
    cv2.destroyAllWindows()

# Call the function with your video path
process_video("video.mp4")
